# Фичи по PTLS   

Цель: предсказать для каждого пользователя взятие/ невзятие каждого из четырех продуктов **в течение месяца после отчетной даты**, исторические данные по ним находятся в targets

In [24]:
import numpy as np

import pandas as pd
from pandas.api.types import is_float_dtype, is_integer_dtype

from collections import Counter
from sklearn.utils import resample

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

import math

from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity

import gc
import glob
import pyarrow.parquet as pq
from tqdm import trange, tqdm

In [25]:
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.decomposition import PCA
from scipy.spatial.distance import pdist, squareform

In [26]:
from typing import List, Optional, Tuple

In [27]:
import warnings

warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=UserWarning, module='pandas')

In [28]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
# найтройки
# Убираем ограничение отображемых колонок
pd.set_option("display.max_columns", None)
# Устанавливаем тему по умолчанию
sb_dark = sns.dark_palette('skyblue', 8, reverse=True) # teal
sns.set(palette=sb_dark)

In [29]:
# Включаем tqdm для pandas, чтобы можно было запускать progress_apply() вместо простого apply()
tqdm.pandas() 
pd.options.display.max_columns = None
pd.options.display.max_rows = 200

In [30]:
eps = 1e-6

In [31]:
PATH = ''
PATH_DATASET_OUTPUT = PATH + 'datasets/'
PATH_DATASET_PTLS = PATH_DATASET_OUTPUT + 'ptls/'

In [167]:
ptls_type = 'trx'
ptls_type = 'geo'
ptls_filename = f'{ptls_type}_emb_select_1_v1.parquet'
ptls_filename

'geo_emb_select_1_v1.parquet'

In [168]:
trx_emb_PTLS_df = pd.read_parquet(PATH_DATASET_PTLS + ptls_filename)
trx_emb_PTLS_df['report_next_end'] = pd.to_datetime(trx_emb_PTLS_df['report_next_end'])
# trx_emb_PTLS_df['trx_date'] = trx_emb_PTLS_df['report_next_end'] - pd.DateOffset(days=45)
trx_emb_PTLS_df = trx_emb_PTLS_df.rename(columns={'emb_trx': 'ptls_embedding'})
trx_emb_PTLS_df.shape

(396434, 3)

In [169]:
trx_emb_PTLS_df

,client_id,report_next_end,ptls_embedding
3,004c7779a0ecbcd972a24627f32dcbbd4d63c610a9ee0a...,2022-11-30,"[-0.07313454, -0.23789532, 0.07221676, -0.8108..."
20,01703640603e9dfa59cd76cc233d086ecc34d3ece32c56...,2022-11-30,"[0.03763936, -0.15345033, 0.046564437, -0.0004..."
23,0191f3d77d4c8f28ad676fe2e1fda9e3c7a9dc0351f0d0...,2022-11-30,"[0.0062002423, -0.23049307, -0.001009638, -0.3..."
29,01df36c22b9fd7e8ad8d7b1c396b2b1367a2606e5b6255...,2022-11-30,"[0.12923148, -0.14685392, 0.015514831, -0.4604..."
40,0292402a3043ce0dc6335c9461a7aa68fbc88d8a49e63d...,2022-11-30,"[0.09086008, -0.24302137, 0.047183976, 0.38375..."
...,...,...,...
788887,f556afdbb6a63a7bedab03ec5abbd3e04980ec76c60e8e...,2022-12-30,"[-0.02733832, -0.05947024, 0.046486262, -0.062..."
788918,f7811e306436328798b252bb698a0affc552304c3272fe...,2022-12-30,"[-0.0011212913, -0.253754, 0.046030834, -0.275..."
788942,f8ebd0078c17acad1d9bfc9bc87c890f742529e447c90d...,2022-12-30,"[0.075992145, 0.38542974, 0.0980213, -0.807597..."
789023,fe3f567e6cb1803ca215ebe293b650ab6ccf25561cbafa...,2022-12-30,"[0.0020023594, -0.4354954, 0.052867875, 0.0492..."


In [170]:
# %%time
# # Переформатируем данные по эмбеддингам
# emb_size = 256
# emb_cols = [f'embed_{i}' for i in range(emb_size)]
# # emb_cols = trx_emb_PTLS_df.columns[:emb_size]
# trx_emb_PTLS_df['trx_embedding'] = trx_emb_PTLS_df.apply(lambda x: x[emb_cols].values, axis=1)
# trx_emb_PTLS_df = trx_emb_PTLS_df[['client_id', 'mon', 'trx_embedding']]
# gc.collect()
# trx_emb_PTLS_df.shape

In [171]:
# # trx_emb_PTLS_df['report_next_end'] = 
# report_next_end = ['2023-01-31', '2022-11-30', '2022-12-30']
# trx_emb_PTLS_df['report_next_end'] = np.random.choice(list(report_next_end), len(trx_emb_PTLS_df))
# trx_emb_PTLS_df

In [172]:
# trx_emb_PTLS_df['ptls_embedding'].apply(lambda x: np.min(x, axis=0))

### Разбивка эмбеддинга на подгруппы и рассчет статистик отдельно для них 

In [173]:
%%time
length_embedding = 256
part_size = 16 # должно быть одно из делителей 256: 1, 2, 4, 8, 16, 32, 64, 128 и 256
count_parts_emb = math.ceil(length_embedding/part_size)
columns_part = []
for i in range(count_parts_emb):
    columns_part.append(f'{i+1}_part_emb')
len(columns_part)

def split_emb(x):
    split_embeddings = [x[i*part_size:(i+1)*part_size] for i in range(count_parts_emb)]
    return split_embeddings
trx_emb_PTLS_df[columns_part] = trx_emb_PTLS_df['ptls_embedding'].apply(lambda x: pd.Series(split_emb(x)))
trx_emb_PTLS_df.shape

CPU times: total: 19.3 s
Wall time: 19.3 s


(396434, 19)

### Рассчет статистик для эмбеддингов PTLS

In [174]:
def calc_emb_stats(data, col):
    data[f'min_{col}'] = data[col].apply(lambda x: np.min(x, axis=0))
    data[f'max_{col}'] = data[col].apply(lambda x: np.max(x, axis=0))
    data[f'sum_{col}'] = data[col].apply(lambda x: np.sum(x, axis=0))
    data[f'std_{col}'] = data[col].apply(lambda x: np.std(x, axis=0))
    data[f'mean_{col}'] = data[col].apply(lambda x: np.mean(x, axis=0))    
    data[f'median_{col}'] = data[col].apply(lambda x: np.median(x, axis=0))    
    return data

In [175]:
%%time
# Формируем фичи по таргету
begin_date = datetime(2022, 1, 1, 0, 0, 0)
start_date = datetime(2022, 1, 1, 0, 0, 0)

end_date = datetime(2023, 1, 31, 0, 0, 0)

# Бланк-датафрейм с клиентами 
# uniq_clients_df = trx_emb_PTLS_df[['client_id']].drop_duplicates()

# Итоговый датасет 
union_ptls_trx_agg_df = pd.DataFrame()

# Бежим по месяцам и расчитываем статистики для клиента берем предыдущие месяцы
for i in trange(((end_date - start_date).days//30 + 1)):
    end_date = start_date + relativedelta(months=1) - relativedelta(days=1)
    print(f'start: {start_date}, end: {end_date}')    
    report_next_end = start_date + relativedelta(months=2) - relativedelta(days=1)
    
#     select_mon_current_df = trx_emb_PTLS_df[trx_emb_PTLS_df['event_time'].between(start_date, end_date)]
    select_mon_current_df = trx_emb_PTLS_df[trx_emb_PTLS_df['report_next_end'] == report_next_end]
    # select_mon_full_df = all_dialogs_df[all_dialogs_df['event_time'].between(begin_date, end_date)]
    print(select_mon_current_df.shape)
    
    client_agg_df = select_mon_current_df[['client_id']].drop_duplicates().copy()
    client_agg_df['report_next_end'] = report_next_end
    client_agg_df = client_agg_df.set_index('client_id')    
    print(client_agg_df.shape)
    
    # Считаем статистики только за прошедший месяц
    stats_mon = calc_emb_stats(data=select_mon_current_df[['client_id', 'report_next_end', 'ptls_embedding']], col='ptls_embedding')
    stats_mon = stats_mon.set_index(['client_id', 'report_next_end'])
    # print(stats_mon.shape)
    client_agg_df = client_agg_df.merge(stats_mon.add_prefix('ptls_trx_'), left_index=True, right_index=True, how='left')
    for col_part in columns_part:
        # print(col_part)
        stats_mon = calc_emb_stats(data=select_mon_current_df[['client_id', 'report_next_end', col_part]], col=col_part)
        stats_mon = stats_mon.set_index(['client_id', 'report_next_end'])
        client_agg_df = client_agg_df.merge(stats_mon.add_prefix(f'ptls_{ptls_type}_{col_part}'), left_index=True, right_index=True, how='left')
        
    
    union_ptls_trx_agg_df = pd.concat([union_ptls_trx_agg_df, client_agg_df])
    start_date = start_date + relativedelta(months=1)
union_ptls_trx_agg_df.shape    

  0%|          | 0/14 [00:00<?, ?it/s]

start: 2022-01-01 00:00:00, end: 2022-01-31 00:00:00
(29411, 19)
(29411, 1)


  7%|▋         | 1/14 [00:19<04:10, 19.26s/it]

start: 2022-02-01 00:00:00, end: 2022-02-28 00:00:00
(35764, 19)
(35764, 1)


 14%|█▍        | 2/14 [00:43<04:22, 21.90s/it]

...

 93%|█████████▎| 13/14 [04:15<00:15, 15.70s/it]

start: 2023-02-01 00:00:00, end: 2023-02-28 00:00:00
(0, 19)
(0, 1)


100%|██████████| 14/14 [04:15<00:00, 18.24s/it]

CPU times: total: 4min 14s
Wall time: 4min 15s


(377469, 120)

In [176]:
union_ptls_trx_agg_df = union_ptls_trx_agg_df.drop(columns=['report_next_end'], errors='ignore')
union_ptls_trx_agg_df.shape

(377469, 119)

In [177]:
union_ptls_trx_agg_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 377469 entries, ('007dde6874ae2cf31e1fa5d1ffd21d57c11d36f29acb0bb765377cdd9716d91a', Timestamp('2022-02-28 00:00:00')) to ('ff6d8e1147bb38af8fe15649a4e749fe5537ad898a1d99f89c7f4d8755bc2e23', Timestamp('2023-01-31 00:00:00'))
Columns: 119 entries, ptls_trx_ptls_embedding to ptls_geo_16_part_embmedian_16_part_emb
dtypes: float32(102), object(17)
memory usage: 207.9+ MB


In [178]:
union_ptls_trx_agg_df

,,ptls_trx_ptls_embedding,ptls_trx_min_ptls_embedding,ptls_trx_max_ptls_embedding,ptls_trx_sum_ptls_embedding,ptls_trx_std_ptls_embedding,ptls_trx_mean_ptls_embedding,ptls_trx_median_ptls_embedding,ptls_geo_1_part_emb1_part_emb,ptls_geo_1_part_embmin_1_part_emb,ptls_geo_1_part_embmax_1_part_emb,ptls_geo_1_part_embsum_1_part_emb,ptls_geo_1_part_embstd_1_part_emb,ptls_geo_1_part_embmean_1_part_emb,ptls_geo_1_part_embmedian_1_part_emb,ptls_geo_2_part_emb2_part_emb,ptls_geo_2_part_embmin_2_part_emb,ptls_geo_2_part_embmax_2_part_emb,ptls_geo_2_part_embsum_2_part_emb,ptls_geo_2_part_embstd_2_part_emb,ptls_geo_2_part_embmean_2_part_emb,ptls_geo_2_part_embmedian_2_part_emb,ptls_geo_3_part_emb3_part_emb,ptls_geo_3_part_embmin_3_part_emb,ptls_geo_3_part_embmax_3_part_emb,ptls_geo_3_part_embsum_3_part_emb,ptls_geo_3_part_embstd_3_part_emb,ptls_geo_3_part_embmean_3_part_emb,ptls_geo_3_part_embmedian_3_part_emb,ptls_geo_4_part_emb4_part_emb,ptls_geo_4_part_embmin_4_part_emb,ptls_geo_4_part_embmax_4_part_emb,ptls_geo_4_part_embsum_4_part_emb,ptls_geo_4_part_embstd_4_part_emb,ptls_geo_4_part_embmean_4_part_emb,ptls_geo_4_part_embmedian_4_part_emb,ptls_geo_5_part_emb5_part_emb,ptls_geo_5_part_embmin_5_part_emb,ptls_geo_5_part_embmax_5_part_emb,ptls_geo_5_part_embsum_5_part_emb,ptls_geo_5_part_embstd_5_part_emb,ptls_geo_5_part_embmean_5_part_emb,ptls_geo_5_part_embmedian_5_part_emb,ptls_geo_6_part_emb6_part_emb,ptls_geo_6_part_embmin_6_part_emb,ptls_geo_6_part_embmax_6_part_emb,ptls_geo_6_part_embsum_6_part_emb,ptls_geo_6_part_embstd_6_part_emb,ptls_geo_6_part_embmean_6_part_emb,ptls_geo_6_part_embmedian_6_part_emb,ptls_geo_7_part_emb7_part_emb,ptls_geo_7_part_embmin_7_part_emb,ptls_geo_7_part_embmax_7_part_emb,ptls_geo_7_part_embsum_7_part_emb,ptls_geo_7_part_embstd_7_part_emb,ptls_geo_7_part_embmean_7_part_emb,ptls_geo_7_part_embmedian_7_part_emb,ptls_geo_8_part_emb8_part_emb,ptls_geo_8_part_embmin_8_part_emb,ptls_geo_8_part_embmax_8_part_emb,ptls_geo_8_part_embsum_8_part_emb,ptls_geo_8_part_embstd_8_part_emb,ptls_geo_8_part_embmean_8_part_emb,ptls_geo_8_part_embmedian_8_part_emb,ptls_geo_9_part_emb9_part_emb,ptls_geo_9_part_embmin_9_part_emb,ptls_geo_9_part_embmax_9_part_emb,ptls_geo_9_part_embsum_9_part_emb,ptls_geo_9_part_embstd_9_part_emb,ptls_geo_9_part_embmean_9_part_emb,ptls_geo_9_part_embmedian_9_part_emb,ptls_geo_10_part_emb10_part_emb,ptls_geo_10_part_embmin_10_part_emb,ptls_geo_10_part_embmax_10_part_emb,ptls_geo_10_part_embsum_10_part_emb,ptls_geo_10_part_embstd_10_part_emb,ptls_geo_10_part_embmean_10_part_emb,ptls_geo_10_part_embmedian_10_part_emb,ptls_geo_11_part_emb11_part_emb,ptls_geo_11_part_embmin_11_part_emb,ptls_geo_11_part_embmax_11_part_emb,ptls_geo_11_part_embsum_11_part_emb,ptls_geo_11_part_embstd_11_part_emb,ptls_geo_11_part_embmean_11_part_emb,ptls_geo_11_part_embmedian_11_part_emb,ptls_geo_12_part_emb12_part_emb,ptls_geo_12_part_embmin_12_part_emb,ptls_geo_12_part_embmax_12_part_emb,ptls_geo_12_part_embsum_12_part_emb,ptls_geo_12_part_embstd_12_part_emb,ptls_geo_12_part_embmean_12_part_emb,ptls_geo_12_part_embmedian_12_part_emb,ptls_geo_13_part_emb13_part_emb,ptls_geo_13_part_embmin_13_part_emb,ptls_geo_13_part_embmax_13_part_emb,ptls_geo_13_part_embsum_13_part_emb,ptls_geo_13_part_embstd_13_part_emb,ptls_geo_13_part_embmean_13_part_emb,ptls_geo_13_part_embmedian_13_part_emb,ptls_geo_14_part_emb14_part_emb,ptls_geo_14_part_embmin_14_part_emb,ptls_geo_14_part_embmax_14_part_emb,ptls_geo_14_part_embsum_14_part_emb,ptls_geo_14_part_embstd_14_part_emb,ptls_geo_14_part_embmean_14_part_emb,ptls_geo_14_part_embmedian_14_part_emb,ptls_geo_15_part_emb15_part_emb,ptls_geo_15_part_embmin_15_part_emb,ptls_geo_15_part_embmax_15_part_emb,ptls_geo_15_part_embsum_15_part_emb,ptls_geo_15_part_embstd_15_part_emb,ptls_geo_15_part_embmean_15_part_emb,ptls_geo_15_part_embmedian_15_part_emb,ptls_geo_16_part_emb16_part_emb,ptls_geo_16_part_embmin_16_part_emb,ptls_geo_16_part_embmax_16_part_emb,ptls_geo_16_part_embsum_16_part_emb,ptls_geo

In [179]:
%%time
# Сохраняем фичи по эмбеддингам ptls
output_filename = PATH_DATASET_OUTPUT + f'ptls_{ptls_type}_agg_{datetime.now().strftime("%d_%m_%Y__%H_%M")}.parquet'
print(output_filename)
union_ptls_trx_agg_df.to_parquet(output_filename)
union_ptls_trx_agg_df.shape

datasets/ptls_geo_agg_15_06_2024__16_36.parquet
CPU times: total: 10.1 s
Wall time: 10.2 s


(377469, 119)